# CS5914 Machine Learning Algorithms
## Assignment 2 
##### Credits: 35% of courework

## Aims

The objectives of this assignment are:

* deepen your understanding of Bayesian inference
* deepen your understanding of clustering and unsupervised learning
* gain experience in implementing sampling-based Bayesian inference algorithms
* gain experience in implementing clustering algorithms


## Set-up

Load required packages (you can only use the imported packages).

In [ ]:
# if you use jupyter-lab, switch to %matplotlib inline instead
# %matplotlib inline
%matplotlib notebook
%config Completer.use_jedi = False
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from scipy.special import logsumexp
import numpy.linalg as linalg
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score

In [ ]:
# use fixed random number generator seed to have reproducible results
random_seed = 123
rng = np.random.default_rng(random_seed)

## Task 1. Spherical K-means

### Read-in datasets
We are going to use two datasets, `dataset1` and `dataset2`. Read in dataset1:
* ``dataset1``: 500 observations and each with 4 features
* note that no cluster labels given for this dataset

In [ ]:
# read in dataset1
d1_df = pd.read_csv('./datasets/dataset1.csv', header=None)
dataset1 = np.array(d1_df)
# it should contain 500 observations and each with 4 dimensional input
n_dataset1, d_dataset1 = dataset1.shape

Read in dataset2, which contains 
* ``dataset2``: 500 observations; and each with 3000 features
* ``dataset2_labels``: the labels are listed in the last column; there are $K=4$ clusters

In [ ]:
# read in dataset2
d2_df = pd.read_csv('./datasets/dataset2.csv', header=None)
dataset2, dataset2_labels = d2_df.iloc[:, 0:-1], d2_df.iloc[:, -1]
dataset2 = np.array(dataset2)
dataset2_labels = np.array(dataset2_labels).astype(int)
n_dataset2, d_dataset2 = dataset2.shape

A quick demonstration of clustering performance evaluation
* use Sklearn.KMeans to fit a basic clustering model and 
* evaluate the clustering performance against the true label with normalised mutual information (NMI)

In [ ]:
# evaluate clustering performance by normalised mutual information.
# use Kmeans to learn the clusters: use the first four rows as centroids
km_dataset2 = KMeans(n_clusters=4, init=dataset2[0:4,:]).fit(dataset2)
# should expect approximately < 0.35 performance by using Kmeans
normalized_mutual_info_score(km_dataset2.labels_, dataset2_labels)

### Spherical K-means
The first task is to implement a variant of K-means algorithm, which is called *Spherical K-means*. The algorithm follows a similar procedure as K-means but with slightly different assignment and update steps. In the following, we will explain the spherical K-means algorithm. 


**Initialisation step**: Start with randomly selecting $K$ data points as the centroid of $K$ clusters. 

**Assignment step**: *Spherical K-means* assigns a data point to the closest centroid based on *cosine distance* rather than Euclidean distance; specifically, for $i=1,\ldots, n$

$$z^{(i)} \leftarrow \arg\min_{k} \left (1- \frac{\boldsymbol{\mu}_k^\top \mathbf{x}^{(i)} }{\|\boldsymbol{\mu}_k\| \cdot \|\mathbf{x}^{(i)}\|}\right ),$$ where $\boldsymbol{\mu}_k^\top \mathbf{x}^{(i)} = \sum_{j=1}^d \boldsymbol{\mu}_{kj} \cdot \mathbf{x}^{(i)}_{j}$ denotes the inner product and $\|\mathbf{x}\|$ is $L_2$ norm of a vector $\mathbf{x}$: $\|\mathbf{x}\| = \sqrt{\mathbf{x}^\top \mathbf{x}}$.

**Update step**: *Spherical K-means* updates the centroids such that they are unit one vectors; for $k=1,\ldots, K$

$$\boldsymbol{\mu}_k \leftarrow \frac{\sum_{i=1}^n I(z^{(i)} =k) \cdot  \mathbf{x}^{(i)}}{\|\sum_{i=1}^n I(z^{(i)} =k) \cdot \mathbf{x}^{(i)}\|}.$$ Note that after the normalisation step, the centroids $\boldsymbol{\mu}_k$ are norm-one vectors: i.e. $\|\boldsymbol{\mu}_k\| = 1$ for $k=1,\ldots, K$.

**Repeat** the above two steps **until** the total cosine distance loss converges, where the loss is defined as
$$\texttt{loss} = \sum_{i=1}^n \left (1- \frac{\boldsymbol{\mu}_{z^{(i)}}^\top \mathbf{x}^{(i)} }{\|\boldsymbol{\mu}_{z^{(i)}}\| \cdot \|\mathbf{x}^{(i)}\|}\right ).$$



#### Task 1.1 Implementation of Spherical K-means

Your task is to complete the code for *assignment* and *update* steps at the given code template of `sphericalKmeans`.

The method `sphericalKmeans` has

**Inputs**:
* `data`: a $n\times d$ matrix to cluster, i.e. each row of $\texttt{data}$ is one observation $x_i$
* `K`: the number of the clusters
* `tol`: tolerence of error, which is used to check whether the loss has converged so the iteration can stop
* `maxIters`: the maximum number of iterations that is allowed

**Outputs**:
* `losses`: the whole trajectory of losses over the iterations
* `zs`: the clustering labels
* `us`: the learnt $K$ centroids

Feel free to write extra helper methods to make your implementation modularised. You may also rewrite the method altogether as long as your method respect the given mtehod's input/output signature. 

In [ ]:
def sphericalKmeans(data, K=3, tol= 1e-4, maxIters= 100):
    n, d = data.shape 
    losses = []
    # initialisation: randomly assign K observations as centroids
    # feel free to use a different but sensible initialisation method    
    init_us_ids = rng.integers(n, size = K)
    us = data[init_us_ids, :]
    zs = np.zeros(n)
    # loop until converge 
    # for i in range(maxIters):
        # assignment step

        # update step
        # convergence check        
    return losses, zs, us

#### Task 1.2 Evaluation

Run your implemented algorithm on `dataset1` with $K =3$. Note that like K-means, Spherical K-means also suffers from bad initialisations. To deal with that, we usually run the algorithm multiple times with different random initialisations. To make your life easier, you may want to write a wrapper method that does it automatically.

Please report the following information based on your results: 
* the learned 3 centroids
* and also plot the loss trajectory.

If you run multiple times, you only need to report the results for the best one.

In [ ]:
# run spherical K-means on dataset1 and report your results here
# use more cells if you need to

Now run your implemented algorithm on `dataset2` with $K =4$. And report the normalised mutual information between the returned cluster labels and true labels.

In [ ]:
# run spherical K-means on dataset2 and report your NMI results here
# use more cells if you need to

## Task 2. Bayesian Machine Learning


In this task, we are going to implement a simple MCMC algorithm to do Bayesian logistic regression. You will be guided to finish the task step by step. First, we read-in the dataset required for the task.

### Dataset
We are going to use the following simple dataset for this task. The dataset is read-in below and stored in variables `XX` and `yy`. There are in total 16 training instances.


In [ ]:
d3_df = pd.read_csv('./datasets/dataset3.csv', header=0)
dataset3 = np.array(d3_df)
XX, yy = dataset3[:, 0:2], dataset3[:, -1]
XX = np.column_stack((np.ones(XX.shape[0]), XX))

### Task 2.1

The log-likelihood for the probabilistic logistic regression is defined as 

$$p(y^{(i)}|\mathbf{w}, \mathbf{x}^{(i)}) = (\sigma^{(i)})^{y^{(i)}}(1-\sigma^{(i)})^{1-y^{(i)}},$$

where $\sigma^{(i)}(\mathbf{x}, \mathbf{w}) = \frac{1}{1+e^{-\mathbf{w}^\top\mathbf{x}}}$ is the logistic regression's output. For simplicity, we assume the feature $\mathbf{x}$ vector includes dummy one and the weight parameter includes the bias.

Give the expression for the joint log-likelihood $\ln p(\{y^{(i)}\}|\mathbf{w}, \mathbf{X})$ for the probabilistic model, where $\mathbf{X}$ is the design matrix ($\mathbf{X} = \begin{bmatrix}\mathbf{x}^{(1)}& \mathbf{x}^{(2)} & \ldots & \mathbf{x}^{(n)}\end{bmatrix}^\top$). 

#### Answer:

#### Implement the joint likelihood method below:

In [ ]:
def log_likelihood(w, X=XX, y=yy):
    return 0

### Task 2.2

A Bayesian logistic regression model usually assumes a zero mean Gaussian prior for the weight parameter $\mathbf{w}$:

$$p(\mathbf{w}|\lambda) = \mathcal{N}_m\left(\mathbf{w}; \mathbf{0}, \frac{1}{\lambda}\mathbf{I}\right)=\prod_{d=1}^m \mathcal{N}(w_d; 0, 1/\lambda)$$

* where $\mathcal{N}_m$ denotes a $m-$dimensional Gaussian distribution and where $\lambda$ is the precision parameter of the Gaussian.

#### Write down an expression for the log-transformed prior $\ln p(\mathbf{w}|\lambda)$

#### Answer:

### Task 2.3

Based on Bayes' rule, the unnormalised log posterior distribution is defined as

$$\ln p(\mathbf{w}|\{y^{(i)}\}, \mathbf{X}) = \ln p(\mathbf{w}|\lambda) + \ln p(\{y^{(i)}\}|\mathbf{X}, \mathbf{w}) +\text{const}$$

#### Write down an expression for the unnormalised log-transformed posterior:

#### Answer:

#### Implement the unnormalised log posterior below:

In [ ]:
def log_posterior(w, X=XX, y=yy, lambda0=1e-2):
    return 0

#### Report the unnormalised log posterior value at $\mathbf{w} =\mathbf{0}$ and $\lambda = \frac{1}{100}$

In [ ]:
log_posterior(np.zeros(XX.shape[1]), XX, yy, lambda0=1e-2)

### Task 2.4

Implement a **Metropolis sampling** algorithm to draw samples from the unnormalised posterior:

$$\mathbf{w}^{(m)} \sim p(\mathbf{w}|\{y^{(i)}\},\mathbf{X}),\;\; \text{for }m = 1, \ldots, M$$

You should use the following proposal distribution

$$q(\mathbf{w}'|\mathbf{w}) = \mathcal{N}\left (\mathbf{w}, \left(\lambda \mathbf{I} + \frac{6}{\pi^2} \mathbf X^\top \mathbf X\right)^{-1}\right),$$

You should convince yourself that the proposal distribution is symmetric. To draw random samples from multivariate Gaussian distribution, you may use `rng.multivariate_normal(mean, cov, size=800)`, where `mean` and `cov` are the corresponding mean and covariance parameters.

In [ ]:
def metropolis_bayes_logistic_reg(X=XX, y=yy, lambda0=1e-2, mc=5000, burnin=1000):
    return w_samples

##### Implement a Monte Carlo prediction method and use the method to replicate the following decision boundary plot. Note that the Monte Carlo prediction is defined as:

$$p(y_{test}|\mathbf{x}_{test}) \approx \frac{1}{M} \sum_{m=1}^M \sigma(\mathbf{x}_{test}, \mathbf{w}^{(m)})$$

You only need roughly $M=1000$ Monte Carlo samples to make accurate predictions.

##### Explain why the Bayesian prediction should be preferred in comparison with the frequentist's MAP/ML estimation.

In [ ]:
def prediction_mc(xtest, wsamples):
    return True

In [ ]:
## replicate the plot below

![Image of Yaktocat](https://leo.host.cs.st-andrews.ac.uk/figs/bayesian_preds.svg);

### Task 2.5 (extension)

Lastly, implement a **Metropolis-Hasting sampling** algorithm to draw samples from the unnormalised posterior:

$$\mathbf{w}^{(m)} \sim p(\mathbf{w}|\{y^{(i)}\},\mathbf{X}),\;\; \text{for }m = 1, \ldots, M$$

You should use the following proposal distribution

$$q(\mathbf{w}') = \mathcal{N}\left (\mathbf{w}_{MAP}, \left(10^2 \cdot \mathbf{I} + \frac{6}{\pi^2} \mathbf X^\top \mathbf X\right)^{-1}\right),$$

* where $\mathbf{w}_{MAP}$ is the maximum aposteriori estimator (MAP), which can be estimated by gradient descent. Note that the proposal is no longer symmetric, therefore you need to use the Metropolis Hasting algorithm instead.

In [ ]:
def metropolis_hasting_bayes_logistic_reg(X=XX, y=yy, lambda0=1e-2, mc=5000, burnin=1000):
    return w_samples

Reuse the prediction method above to replicate the prediction plot.

## Submission
Hand in via Moodle: the completed jupyter notebook.



## Marking
Your submission will be marked as a whole. 

* to get a grade above 13, you are expected to finish at least Task 1 to a good standard
* to get a grade above 13 and up to 17, you are expected to answer Task 1 and Task 2.1-2.3 to a good standard
* to achieve a grade of 17-18, you are expected to finish all tasks except Task 2.5 flawlessly 
* to get 18+, you are expected to attempt all questions flawlessly


Marking is according to the standard mark descriptors published in the Student Handbook at:

https://info.cs.st-andrews.ac.uk/student-handbook/learning-teaching/feedback.html#GeneralMarkDescriptors


You must reference any external sources used. Guidelines for good academic practice are outlined in the student handbook at https://info.cs.st-andrews.ac.uk/student-handbook/academic/gap.html


## Submission dates
There are no fixed submission dates. Formal exam boards are held in January, May and September. Any provisional grades recorded on MMS are discussed by the Programme team and the External Examiner. Credits for a module can only be obtained after all the coursework has been discussed at an exam board, so students must submit work at least three weeks before the board date, giving time for grading and preparation of feedback.

https://www.st-andrews.ac.uk/education/staff/assessment/reporting/